# VF2Q

In [1]:
from ag import *
from vf2q import VF2Q
from qiskit.circuit import QuantumCircuit
from networkx import Graph
from numpy import inf

import csv
import os
import time

In [3]:
def run(arch: Graph, circ: QuantumCircuit, reps: int) -> dict:

    VF2Q.archgraph = arch
    VF2Q.circuit = circ

    min_runtime = inf
    min_subcircs = inf
    min_swaps = inf
    min_depth = inf
    min_gates = inf
    best_circ = None

    for _ in range(reps):

        start = time.time()
        VF2Q.partition_circuit()
        VF2Q.match_subcircs(w1=0.1, w2=0.1)
        VF2Q.apply_swaps()
        VF2Q.transform_circuit(decompose=True)
        end = time.time()

        # objective: depth
        if VF2Q.transformed_circ.depth() < min_depth:
            min_runtime = end - start
            min_subcircs = len(VF2Q.subcircs)
            min_swaps = sum([len(swaps) for swaps in VF2Q.swaps])
            min_depth = VF2Q.transformed_circ.depth()
            min_gates = len(VF2Q.transformed_circ)
            best_circ = VF2Q.transformed_circ
        
    return {
        "min_runtime": min_runtime,
        "min_subcircs": min_subcircs,
        "min_swaps": min_swaps,
        "min_depth": min_depth,
        "min_gates": min_gates,
        "best_circ": best_circ
    }

In [ ]:
ARCH = tokyo()
BENCH = "benchmark/qiskit_bench/"
MAX_NUM_QUBITS = 16
REPS = 1

csv_file = open("vf2q_data1.csv", "w", newline="")
writer = csv.writer(csv_file)
writer.writerow([
    "circuit",
    "#subcircuits",
    "#swaps",
    "depth_overhead",
    "gate_overhead",
    "runtime"
])

for file in os.listdir(BENCH):

    circ_name = file.rstrip(".qasm")
    
    if int(circ_name.split('_')[-1]) > MAX_NUM_QUBITS:
        continue

    circ = QuantumCircuit.from_qasm_file(BENCH + file)
    res = run(ARCH, circ, REPS)

    print(circ_name)
    print(f"runtime: {res['min_runtime']}")
    print(f"#subcircuits: {VF2Q.circuit.depth()} -> {res['min_subcircs']} ({res['min_subcircs'] - VF2Q.circuit.depth()})")
    print(f"#swaps: {res['min_swaps']}")
    print(f"#depth: {VF2Q.circuit.depth()} -> {res['min_depth']} (+{res['min_depth'] - VF2Q.circuit.depth()})")
    print(f"#gates: {len(VF2Q.circuit)} -> {res['min_gates']} (+{res['min_gates'] - len(VF2Q.circuit)})")
    print()

    writer.writerow([
        circ_name,
        res["min_subcircs"],
        res["min_swaps"],
        res["min_depth"] - VF2Q.circuit.depth(),
        res["min_gates"] - len(VF2Q.circuit),
        res["min_runtime"]
    ])

csv_file.close()